In [1]:
import numpy as np
import os 
import pandas as pd
def get_signal_length(signal):
      return len(signal)

In [ ]:
def classify_grasp(grasp_values):
    grasp_sum = np.sum(grasp_values)
    grasp_abs = np.sum(np.abs(grasp_values))
    if (grasp_sum == grasp_abs) and grasp_abs != 0 :
        return "Positive movement"
    elif (grasp_sum == -grasp_abs) and grasp_abs != 0 :
        return "Negative movement"
    elif (grasp_abs == 0):
        return "No movement"
    else:
        return None


def classify_rotation(rotation_values):
    rotation_sum = np.sum(rotation_values)
    rotation_abs = np.sum(np.abs(rotation_values))
    if (rotation_sum == rotation_abs) and rotation_abs != 0:
        return "Positive movement"
    elif (rotation_sum == -rotation_abs) and rotation_abs != 0:
        return "Negative movement"
    elif (rotation_abs == 0):
        return "No movement"
    else:
        return None
    
def process_data(df):
    df["window"] = (df.index // 12).astype(int)
    grouped = df.groupby("window").agg({
        "Z1": list,
        "Z2": list,
        "grasp": list,
        "rotation": list,
        "abs_coef1": list,
        "abs_coef2": list
    }).reset_index()

    grouped["Z1_fft"] = grouped["Z1"].apply(lambda x: list(np.abs(np.fft.fft(x))))
    grouped["Z2_fft"] = grouped["Z2"].apply(lambda x: list(np.abs(np.fft.fft(x))))

    grouped["label_grasp"] = grouped.apply(lambda row: classify_grasp(row["grasp"]), axis=1)
    grouped["label_rotation"] = grouped.apply(lambda row: classify_rotation(row["rotation"]), axis=1)

    grouped = grouped.drop(columns=["grasp", "rotation", "abs_coef1", "abs_coef2"])

    return grouped

file_list = pd.read_csv("Your_DataFrame.csv", encoding='utf-8')  


all_features = []

for _, row in file_list.iterrows():
    file_path = row["file_path"]
    
    if not os.path.exists(file_path):
        continue
    
    
    df = pd.read_excel(file_path)
    processed_df = process_data(df)
    
    processed_df["file_name"] = os.path.basename(file_path)
    all_features.append(processed_df)


final_df = pd.concat(all_features, ignore_index=True)
print(final_df.head())
final_df.to_excel("Your_Final_Data_Frame", index=False)